In [2]:
pip install pydicom

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 660.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.4 MB 975.2 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.4 MB 2.0 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.4 MB 2.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.4 MB 2.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.4 MB 2.5 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.4 MB 2.7 MB/s eta 0:00:01
   --------------- ------------------------ 0.9/2.4 MB 2.6 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 2.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.2/2.4 MB 2.8 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.4 MB 2.7 MB/s eta 0:00:01
   ----------------------- ---------------- 1.4/2.4 MB 2.7 MB/s eta 0:00:01
   ------------

In [1]:
# David Ouyang 10/2/2019

# Notebook which iterates through a folder, including subfolders, 
# and convert DICOM files to AVI files of a defined size (natively 112 x 112)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math

destinationFolder = "processed_videos"


In [10]:
# Dependencies you might need to run code
# Commonly missing

#!pip install pydicom
#!pip install opencv-python


In [2]:
def mask(output):
    dimension = output.shape[0]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))

    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = np.reshape(mask, (dimension, dimension)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage

In [3]:
def makeVideo(fileToProcess, destinationFolder):
    try:
        fileName = fileToProcess.split('\\')[-1] #\\ if windows, / if on mac or sherlock
                                                 #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()

        if not os.path.isdir(os.path.join(destinationFolder,fileName)):

            dataset = dicom.dcmread(fileToProcess, force=True)
            testarray = dataset.pixel_array

            frame0 = testarray[0]
            mean = np.mean(frame0, axis=1)
            mean = np.mean(mean, axis=1)
            yCrop = np.where(mean<1)[0][0]
            testarray = testarray[:, yCrop:, :, :]

            bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
            if bias>0:
                if testarray.shape[1] < testarray.shape[2]:
                    testarray = testarray[:, :, bias:-bias, :]
                else:
                    testarray = testarray[:, bias:-bias, :, :]


            print(testarray.shape)
            frames,height,width,channels = testarray.shape

            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")

            fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
            video_filename = os.path.join(destinationFolder, fileName + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)


            for i in range(frames):

                outputA = testarray[i,:,:,0]
                smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

                # Resize image
                output = cv2.resize(smallOutput, cropSize, interpolation = cv2.INTER_CUBIC)

                finaloutput = mask(output)


                finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
                out.write(finaloutput)

            out.release()

        else:
            print(fileName,"hasAlreadyBeenProcessed")
    except:
        print("something filed, not sure what, have to debug", fileName)
    return 0

In [6]:
AllA4cNames = "input_videos"

count = 0
    
cropSize = (112,112)
subfolders = os.listdir(AllA4cNames)


for folder in subfolders:
    print(folder)

    for content in os.listdir(os.path.join(AllA4cNames, folder)):
        for subcontent in os.listdir(os.path.join(AllA4cNames, folder, content)):
            count += 1
            

            VideoPath = os.path.join(AllA4cNames, folder, content, subcontent)

            print(count, folder, content, subcontent)

            if not os.path.exists(os.path.join(destinationFolder,subcontent + ".avi")):
                makeVideo(VideoPath, destinationFolder)
            else:
                print("Already did this file", VideoPath)


print(len(AllA4cFilenames))

1 1_3s.mp4
something filed, not sure what, have to debug 1_3s.mp4
Total processed files: 1


In [7]:
import os
import cv2
import numpy as np

def mask(output):
    dimension = output.shape[0]
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))
    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = mask.astype(np.uint8)
    maskedImage = cv2.bitwise_and(output, output, mask=mask)
    return maskedImage

def makeVideo(fileToProcess, destinationFolder):
    try:
        fileName = os.path.splitext(os.path.basename(fileToProcess))[0]

        if not os.path.exists(os.path.join(destinationFolder, fileName + '.avi')):
            cap = cv2.VideoCapture(fileToProcess)
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()

            if len(frames) == 0:
                print("No frames found in video:", fileName)
                return

            fps = cap.get(cv2.CAP_PROP_FPS)
            if fps == 0 or fps is None:
                fps = 30  # 默认帧率

            height, width, channels = frames[0].shape

            fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
            video_filename = os.path.join(destinationFolder, fileName + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)

            for frame in frames:
                # 裁剪和调整大小
                h_crop = int(height * 0.1)
                w_crop = int(width * 0.1)
                cropped_frame = frame[h_crop:height - h_crop, w_crop:width - w_crop]

                output = cv2.resize(cropped_frame, cropSize, interpolation=cv2.INTER_CUBIC)
                finaloutput = mask(output)
                finaloutput = cv2.merge([finaloutput, finaloutput, finaloutput])
                out.write(finaloutput)

            out.release()
            print(f"Processed video: {fileName}")

        else:
            print(fileName, "has already been processed")
    except Exception as e:
        print("An error occurred:", e)
    return 0

AllA4cNames = "input_videos"
destinationFolder = "output_videos"
if not os.path.exists(destinationFolder):
    os.makedirs(destinationFolder)

count = 0
cropSize = (112,112)

for subcontent in os.listdir(AllA4cNames):
    if subcontent.endswith('.mp4'):  # 只处理 .mp4 文件
        count += 1
        VideoPath = os.path.join(AllA4cNames, subcontent)
        print(count, subcontent)
        if not os.path.exists(os.path.join(destinationFolder, os.path.splitext(subcontent)[0] + ".avi")):
            makeVideo(VideoPath, destinationFolder)
        else:
            print("Already processed this file:", VideoPath)


1 1_3s.mp4
Processed video: 1_3s
